In [1]:
import argparse
import glob
import os
import time
import sys

sys.path.insert(1, './nerf')
os.environ['GPU_DEBUG']='3'
import numpy as np
import torch
import torchvision
import yaml
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

from nerf.load_flame import load_flame_data

from nerf import (CfgNode, get_embedding_function, get_ray_bundle, img2mse,
                  load_llff_data, meshgrid_xy, models,
                  mse2psnr, run_one_iter_of_nerf, dump_rays, GaussianSmoothing)
#from gpu_profile import gpu_profile

In [7]:
config = 'nerface_dataset/person_1/person_1_config.yml'
cfg = None
with open(config, "r") as f:
    cfg_dict = yaml.load(f, Loader=yaml.FullLoader)
    cfg = CfgNode(cfg_dict)
cfg.dataset

CfgNode({'type': 'blender', 'basedir': 'nerface_dataset/person_1', 'half_res': False, 'testskip': 1, 'no_ndc': True, 'near': 0.2, 'far': 0.8})

In [17]:
cfg_dict

{'experiment': {'id': 'person_1',
  'logdir': 'logs/',
  'randomseed': 42,
  'train_iters': 1000000,
  'validate_every': 1000,
  'save_every': 5000,
  'print_every': 100,
  'device': 0},
 'dataset': {'type': 'blender',
  'basedir': 'nerface_dataset/person_1',
  'half_res': False,
  'testskip': 1,
  'no_ndc': True,
  'near': 0.2,
  'far': 0.8},
 'models': {'coarse': {'type': 'ConditionalBlendshapePaperNeRFModel',
   'num_layers': 4,
   'hidden_size': 256,
   'skip_connect_every': 3,
   'include_input_xyz': True,
   'log_sampling_xyz': True,
   'num_encoding_fn_xyz': 10,
   'use_viewdirs': True,
   'include_input_dir': False,
   'num_encoding_fn_dir': 4,
   'log_sampling_dir': True},
  'fine': {'type': 'ConditionalBlendshapePaperNeRFModel',
   'num_layers': 4,
   'hidden_size': 256,
   'skip_connect_every': 3,
   'num_encoding_fn_xyz': 10,
   'include_input_xyz': True,
   'log_sampling_xyz': True,
   'use_viewdirs': True,
   'include_input_dir': False,
   'num_encoding_fn_dir': 4,
   'lo

### `load_flame_data`
`def load_flame_data(basedir, half_res=False, testskip=1, debug=False,expressions=True,load_frontal_faces=False, load_bbox=True, test=False)`  
- basedir: path
- half_res:
- testskip: 



In [10]:
images, poses, render_poses, hwf, expressions = None, None, None, None, None
if cfg.dataset.type.lower() == "blender":
    images, poses, render_poses, hwf, i_split, expressions, _, bboxs = load_flame_data(
        cfg.dataset.basedir,
        half_res=cfg.dataset.half_res,
        testskip=cfg.dataset.testskip,
    )
    i_train, i_val, i_test = i_split
    H, W, focal = hwf
    H, W = int(H), int(W)
    hwf = [H, W, focal]
    if cfg.nerf.train.white_background:
        images = images[..., :3] * images[..., -1:] + (1.0 - images[..., -1:])
print("done loading data")

starting data loading
Done with data loading
done loading data


In [14]:
import json
import os

import cv2
import imageio
import numpy as np
import torch
metas = {}
splits = ["train", "val", "test"]
for s in splits:
    with open(os.path.join(cfg.dataset.basedir, f"transforms_{s}.json"), "r") as fp:
        metas[s] = json.load(fp)

In [16]:
os.path.join(cfg.dataset.basedir, f"transforms_{s}.json")

'nerface_dataset/person_1/transforms_test.json'

In [18]:

print(torch.cuda.device_count())

2
